In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
train = pd.read_csv('./data/train.csv',index_col='index')
train.head()

,event_id,X,Y,Z,TX,TY,chi2,signal,brick_number
index,,,,,,,,,
0,-999,66162.843750,65620.031250,0.0,0.166890,0.276674,2.988714,0.0,44
1,-999,43900.445312,67091.671875,0.0,-0.179967,-0.478795,1.901420,0.0,98
2,-999,37564.019531,26618.890625,55599.0,-0.267820,-0.282087,2.510245,0.0,64
3,-999,39380.027344,48135.425781,19395.0,0.265197,-0.577493,1.215219,0.0,63
4,-999,65040.781250,44018.468750,12930.0,-0.711161,-0.245592,2.973661,0.0,2


In [8]:
# оценим количество кирпичей
train.brick_number.unique().shape

(100L,)

In [37]:
# predict position of the track on the next layer
def getPosition(track,dz):
    '''
    track - track record (X,Y,Z,TX,TY)
    '''
    pos = np.array([track.X, track.Y, track.TX, track.TY])
    pos[0] +=  dz*np.tan(track.TX)
    pos[1] +=  dz*np.tan(track.TY)  
    return pos

In [15]:
brick2 = train.query('brick_number == 2')

In [17]:
brick2.query('signal == 1').event_id.unique()

array([15847, 20217,  4299], dtype=int64)

Примерный план обработки:
    <ul>
    <li> Разобрать треки по кирпичам
    <li> Отсортировать по расстоянию z, т.е. по слоям. Подумать о введении дополнительной переменной "слой"
    <li> Отсортировать по координате X
    <li> Начать перебор с начала кирпича.
    </ul>

In [89]:
idx1 = 3901599
idx2 = 3670692
prediction = getPosition(train.iloc[idx1,:],18102.0-16809.0)

In [133]:
dx = 20 # estimation of error along OX axis
dy = 20 # estimation of error along OY axis
dtx = 0.01
dty = 0.01

In [43]:
# add new feature - layer number

In [66]:
zPos = np.sort(event15847.Z.unique())
zPosDict = dict(enumerate(z)) # dictionary of positions (layerNumber => position)
zDiffDict = dict(enumerate(np.diff(zPos))) # dictionaru of distances (layerNumber => distance to the next layer)
layerDict = {v: k for k, v in zPosDict.iteritems() } # dictionary of layer number (position => layerNumber)

In [78]:
event15847 = event15847.assign(layer = event15847.Z.map(layerDict))

In [87]:
event15847.sort_values(by='layer',ascending=True).head()

,event_id,X,Y,Z,TX,TY,chi2,signal,brick_number,layer
index,,,,,,,,,,
3901599,15847,35281.085938,28812.714844,16809.0,0.096675,0.051116,0.568254,1.0,2,0
3670692,15847,35407.148438,28880.599609,18102.0,0.101562,0.052474,0.318941,1.0,2,1
101836,15847,35535.875000,28950.218750,19395.0,0.104953,0.056952,0.966164,1.0,2,2
2639629,15847,35667.257812,29018.583984,20688.0,0.101516,0.056934,1.357672,1.0,2,3
3443804,15847,35798.773438,29085.591797,21981.0,0.102604,0.048461,1.086284,1.0,2,4


In [100]:
brick = brick2

In [107]:
# create dictionaries
zPos = np.sort(brick.Z.unique())
zPosDict = dict(enumerate(zPos)) # dictionary of positions (layerNumber => position)
zDiffDict = dict(enumerate(np.diff(zPos))) # dictionaru of distances (layerNumber => distance to the next layer)
layerDict = {v: k for k, v in zPosDict.iteritems() } # dictionary of layer number (position => layerNumber)

# add extra feature - layer number
brick = brick.assign(layerId = brick.Z.map(layerDict))

In [125]:
idx1 = 3901599
idx2 = 3670692
layerId = brick.loc[idx1].layerId
prediction = getPosition(brick.loc[idx1],zDiffDict.get(layerId))
prediction

array([  3.54064776e+04,   2.88788655e+04,   9.66750383e-02,
         5.11160716e-02])

In [157]:
brick.loc[idx1]

event_id        15847.000000
X               35281.085938
Y               28812.714844
Z               16809.000000
TX                  0.096675
TY                  0.051116
chi2                0.568254
signal              1.000000
brick_number        2.000000
layerId            13.000000
Name: 3901599, dtype: float64

In [132]:
brick.loc[idx2]

event_id        15847.000000
X               35407.148438
Y               28880.599609
Z               18102.000000
TX                  0.101562
TY                  0.052474
chi2                0.318941
signal              1.000000
brick_number        2.000000
layerId            14.000000
Name: 3670692, dtype: float64

In [139]:
set1 = brick.query('layerId == @layerId+1 & X > @prediction[0]-@dx & X < @prediction[0]+@dx & Y > @prediction[1]-@dy & Y < @prediction[1]+@dy & TX > @prediction[2]-@dtx & TX < @prediction[2]+@dtx & TY > @prediction[3]-@dty & TY < @prediction[2]+@dty')
set1

,event_id,X,Y,Z,TX,TY,chi2,signal,brick_number,layerId
index,,,,,,,,,,
3670692,15847,35407.148438,28880.599609,18102.0,0.101562,0.052474,0.318941,1.0,2,14


In [324]:
layerId = 20
layer0 = brick.query('layerId == @layerId')
layer1 = brick.query('layerId == @layerId+1').sort_values(by='X',ascending=True)

In [344]:
layer0.head()

,event_id,X,Y,Z,TX,TY,chi2,signal,brick_number,layerId
index,,,,,,,,,,
5256,-999,44497.687500,38916.789062,25860.0,0.171391,0.347470,1.445627,0.0,2,20
12492,-999,36494.312500,48617.421875,25860.0,0.442829,-0.479743,2.347063,0.0,2,20
13631,-999,46141.500000,19839.148438,25860.0,0.209003,-0.232413,1.878767,0.0,2,20
14129,-999,43945.972656,36422.921875,25860.0,0.427083,0.381566,2.511479,0.0,2,20
17923,-999,43274.703125,62394.203125,25860.0,-0.067690,-0.559747,0.303846,0.0,2,20


In [351]:
layer0.reset_index().as_matrix(columns = ['index', 'X', 'TX', 'Y', 'TY', 'signal'])

array([[  5.25600000e+03,   4.44976875e+04,   1.71391368e-01,
          3.89167891e+04,   3.47470224e-01,   0.00000000e+00],
       [  1.24920000e+04,   3.64943125e+04,   4.42829251e-01,
          4.86174219e+04,  -4.79743302e-01,   0.00000000e+00],
       [  1.36310000e+04,   4.61415000e+04,   2.09002972e-01,
          1.98391484e+04,  -2.32412577e-01,   0.00000000e+00],
       ..., 
       [  9.79546900e+06,   2.97442109e+04,  -1.47456288e-01,
          5.83007031e+03,  -1.29078314e-01,   0.00000000e+00],
       [  9.80365700e+06,   6.26292539e+04,  -9.96651798e-02,
          6.02760078e+04,   5.04575908e-01,   0.00000000e+00],
       [  9.80456900e+06,   6.62811172e+04,   1.38504460e-01,
          6.56429297e+03,   3.24530333e-01,   0.00000000e+00]])

In [331]:
-

         NaN        TX        TY             X             Y        Z  \
1189703  NaN -0.050288  0.089848  61136.050781  63362.929688  27153.0   
7847358  NaN -0.017294  0.143197  61178.773438  63422.820312  27153.0   
6372224  NaN  0.011184  0.298916  61191.980469  63442.195312  27153.0   
168693   NaN -0.149600  0.175823  60957.359375  63450.710938  27153.0   
6044448  NaN -0.094192  0.603478  61009.382812  64492.781250  27153.0   
514386   NaN  0.206362  0.151297  44936.914062   6058.601562  27153.0   
6044448  NaN -0.094192  0.603478  61009.382812  64492.781250  27153.0   
168693   NaN -0.149600  0.175823  60957.359375  63450.710938  27153.0   
6404684  NaN  0.030915 -0.261486  62396.656250  44048.890625  27153.0   
1918759  NaN  0.010040 -0.288681  62404.691406  43484.105469  27153.0   
2230919  NaN  0.089416  0.071461  36181.250000  29360.632812  27153.0   
486485   NaN  0.104250  0.056752  36315.515625  29368.933594  27153.0   
6372224  NaN  0.011184  0.298916  61191.980469  634

In [336]:
layer1.query('signal == 1.0').head()

,event_id,X,Y,Z,TX,TY,chi2,signal,brick_number,layerId
index,,,,,,,,,,
6968838,15847,35631.117188,29767.648438,27153.0,-0.166815,0.003037,1.388220,1.0,2,21
2230919,15847,36181.250000,29360.632812,27153.0,0.089416,0.071461,0.772666,1.0,2,21
486485,15847,36315.515625,29368.933594,27153.0,0.104250,0.056752,0.457369,1.0,2,21
8812266,15847,37152.511719,30212.210938,27153.0,0.328502,0.440890,1.444820,1.0,2,21
523061,15847,37220.554688,30052.429688,27153.0,0.350670,0.130162,0.691582,1.0,2,21


In [337]:
binRes.head()

,None,TX,TY,X,Y,Z,brick_number,chi2,event_id,layerId,signal
1189703,NaN,-0.050288,0.089848,61136.050781,63362.929688,27153.0,2.0,0.836000,4299.0,21.0,1.0
7847358,NaN,-0.017294,0.143197,61178.773438,63422.820312,27153.0,2.0,0.689521,4299.0,21.0,1.0
6372224,NaN,0.011184,0.298916,61191.980469,63442.195312,27153.0,2.0,1.173308,4299.0,21.0,1.0
168693,NaN,-0.149600,0.175823,60957.359375,63450.710938,27153.0,2.0,0.739837,4299.0,21.0,1.0
6044448,NaN,-0.094192,0.603478,61009.382812,64492.781250,27153.0,2.0,0.639762,4299.0,21.0,1.0


In [355]:
layer0Arr = layer0.reset_index().as_matrix(columns = ['index', 'X', 'Y', 'TX', 'TY', 'signal'])
layer1Arr = layer1.reset_index().as_matrix(columns = ['index', 'X', 'Y', 'TX', 'TY', 'signal'])

In [371]:
# перепишу по массивы
#%%time
result  = pd.DataFrame(columns=layer1.columns.names)
dx  = 100 # estimation of error along OX axis
dy  = 100 # estimation of error along OY axis
dtx = 0.2
dty = 0.2

dz = zDiffDict.get(layerId)
for track in layer0Arr:
    #calculate new pos
    newX  = track[1]+dz*np.tan(track[3])
    newY  = track[2]+dz*np.tan(track[4])
    newTX = track[3]
    newTY = track[4]
    
    #print 
    lEdge = np.searchsorted(layer1Arr[:,1],newX-dx,side='left')
    rEdge = np.searchsorted(layer1Arr[:,1],newX+dx,side='right')
    
    subSet = layer1Arr[lEdge:rEdge,:]
    print subSet
    
    '''
    if False:
        lBorder = layer1['X'].searchsorted(predPosition[0]-dx,side='left')[0]  # find left border
        rBorder = layer1['X'].searchsorted(predPosition[0]+dx,side='right')[0]  # find left border
        if (lBorder == rBorder):
            rBorder+=1

        subSet = layer1.iloc[lBorder:rBorder+1]

        if subSet.shape[0]>0:    
            res = subSet.query('Y > @predPosition[1]-@dy & Y < @predPosition[1]+@dy & TX > @predPosition[2]-@dtx & TX < @predPosition[2]+@dtx & TY > @predPosition[3]-@dty & TY < @predPosition[3]+@dty')
            result = result.append(res)
        
    else:
        q = layer1.query('X > @predPosition[0]-@dx & X < @predPosition[0]+@dx & Y > @predPosition[1]-@dy & Y < @predPosition[1]+@dy & TX > @predPosition[2]-@dtx & TX < @predPosition[2]+@dtx & TY > @predPosition[3]-@dty & TY < @predPosition[3]+@dty')
        if q.shape[0] >0:
            result = result.append(q)
    '''
#print result
#print result.shape

[[  3.66734500e+06   4.46335898e+04   4.36985352e+04   2.77446061e-01
    1.42113091e-02   0.00000000e+00]
 [  5.26487600e+06   4.46671250e+04   7.21228594e+04  -3.08026403e-01
    6.07886910e-02   0.00000000e+00]
 [  2.96885900e+06   4.46730508e+04   1.16013828e+04   3.30803573e-01
   -2.96712250e-01   0.00000000e+00]
 [  5.39442800e+06   4.46772344e+04   2.78272734e+04  -2.69196421e-01
    4.28924859e-01   0.00000000e+00]
 [  2.88202000e+05   4.46948867e+04   5.75947617e+04  -5.33500731e-01
   -2.08407745e-01   0.00000000e+00]
 [  3.35589600e+06   4.46959102e+04   5.50708242e+04  -2.07217261e-01
   -3.28534216e-01   0.00000000e+00]
 [  9.18961900e+06   4.47031719e+04   6.25043125e+04   1.34021580e-01
    4.41071421e-01   0.00000000e+00]
 [  5.02862300e+06   4.47254961e+04   4.30131250e+04  -1.70982137e-01
    7.82738104e-02   0.00000000e+00]
 [  3.87420100e+06   4.47679023e+04   2.52745625e+04   3.37825507e-01
   -4.26990315e-02   0.00000000e+00]
 [  9.58494800e+06   4.47844062e+04  